In [1]:
'''更新日志09July
这是第一个beta版，已经能无bug运行，排版还没定下来，调参不太方便。
体素模型比预期中快得多，比点云还快。
学习率递减还没写，这可能导致loss在下降中遇到瓶颈。
没有配准的程序，所以现在输入的数据是乱的。
'''

'更新日志09July\n这是第一个beta版，已经能无bug运行，排版还没定下来，调参不太方便。\n体素模型比预期中快得多，比点云还快。\n学习率递减还没写，这可能导致loss在下降中遇到瓶颈。\n没有配准的程序，所以现在输入的数据是乱的。\n'

In [2]:
'''
双通道3D卷积神经网络，使用体素模型。相比点云方法，优点是能够直接进行3D卷积运算，缺点是数据量和算量都更大。
通道1：原始牙齿的体素模型
通道2：预备体的体素模型
网络结构：3D卷积-池化-3D卷积-池化-扁平化-MLP-输出
'''

'\n双通道3D卷积神经网络，使用体素模型。相比点云方法，优点是能够直接进行3D卷积运算，缺点是数据量和算量都更大。\n通道1：原始牙齿的体素模型\n通道2：预备体的体素模型\n网络结构：3D卷积-池化-3D卷积-池化-扁平化-MLP-输出\n'

In [3]:
'''
# 安装环境
pip install trimesh
'''

'\n# 安装环境\npip install trimesh\n'

In [4]:
'''
# 安装环境
pip install torch torchvision torchaudio
'''

'\n# 安装环境\npip install torch torchvision torchaudio\n'

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
import trimesh
from sklearn.model_selection import KFold

/home/jupyter-24143/.local/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [6]:
# 加载和归一化模型
def load_mesh(file_path):
    mesh = trimesh.load(file_path)
    # 如果加载的是Scene对象，转换为Trimesh对象
    if isinstance(mesh, trimesh.Scene):
        mesh = mesh.dump(concatenate=True)
    return mesh

def normalize_mesh(mesh):
    # 将网格中心平移到原点
    mesh.apply_translation(-mesh.centroid)
    # 计算缩放比例
    scale_factor = 1.0 / max(mesh.extents)
    # 缩放网格
    mesh.apply_scale(scale_factor)
    return mesh

def mesh_to_voxel(mesh, voxel_size=32):
    # 将网格体素化
    voxelized_mesh = mesh.voxelized(pitch=1.0 / voxel_size)
    # 确保体素矩阵的形状为 (voxel_size, voxel_size, voxel_size)
    voxel_matrix = voxelized_mesh.matrix
    padded_matrix = np.zeros((voxel_size, voxel_size, voxel_size), dtype=voxel_matrix.dtype)
    shape = np.minimum(voxel_matrix.shape, (voxel_size, voxel_size, voxel_size))
    padded_matrix[:shape[0], :shape[1], :shape[2]] = voxel_matrix[:shape[0], :shape[1], :shape[2]]
    return padded_matrix

# 加载并归一化打磨前的初始模型
initial_mesh_path = 'incisor.obj'
initial_mesh = normalize_mesh(load_mesh(initial_mesh_path))
voxel_size = 32  # 体素网格尺寸
voxel_data_initial = mesh_to_voxel(initial_mesh, voxel_size)

class TeethDataset(Dataset):
    def __init__(self, initial_voxel, score_file_path, voxel_size=32):
        self.initial_voxel = initial_voxel
        self.indices, self.scores = read_scores(score_file_path)
        self.voxel_size = voxel_size

    def __len__(self):
        return len(self.scores)

    def __getitem__(self, idx):
        index = self.indices[idx]
        score = self.scores[idx]
        # 加载并归一化打磨后的模型
        mesh_path = f'{index}.obj'
        mesh = normalize_mesh(load_mesh(mesh_path))
        voxel_data_after = mesh_to_voxel(mesh, self.voxel_size)
        # 双通道数据
        sample = np.stack([self.initial_voxel, voxel_data_after], axis=0)
        return torch.tensor(sample, dtype=torch.float32), torch.tensor([score], dtype=torch.float32)  # 注意这里将 score 包装为列表


In [7]:
# 读取评分
def read_scores(score_file_path):
    indices = []
    scores = []
    with open(score_file_path, 'r') as f:
        for line in f:
            index, score = line.strip().split()
            indices.append(int(index))
            scores.append(float(score))
    return indices, scores  

# 创建数据集
score_file_path = 'scores.txt'
dataset = TeethDataset(voxel_data_initial, score_file_path, voxel_size)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)


In [8]:
# 定义模型
class Teeth3DCNN(nn.Module):
    def __init__(self):
        super(Teeth3DCNN, self).__init__()
        self.conv1 = nn.Conv3d(2, 32, kernel_size=3, stride=1, padding=1)  # 双通道输入
        self.conv2 = nn.Conv3d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 8 * 8 * 8, 256)  # 体素尺寸为32*32*32，经过两次pooling变为8*8*8
        self.fc2 = nn.Linear(256, 1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool3d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# 创建模型
model = Teeth3DCNN()
criterion = nn.MSELoss()  # 损失函数
optimizer = optim.Adam(model.parameters(), lr=0.001)  # 学习率和优化器

In [9]:
'''
10epoch实验中每epochloss不稳定，30epoch跑过两次结果都可以
泛化能力未知
'''

# 训练模型
num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, targets in dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(dataloader)}")

# 模型评估
model.eval()
test_loss = 0.0
with torch.no_grad():
    for inputs, targets in dataloader:
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        test_loss += loss.item()
print(f"Test Loss: {test_loss / len(dataloader)}")


Epoch 1/30, Loss: 868.0476494839317
Epoch 2/30, Loss: 286.99375313206724
Epoch 3/30, Loss: 318.74164380525286
Epoch 4/30, Loss: 277.3193368660776
Epoch 5/30, Loss: 363.78813158838375
Epoch 6/30, Loss: 305.28822512375683
Epoch 7/30, Loss: 249.0247506593403
Epoch 8/30, Loss: 255.31298170591654
Epoch 9/30, Loss: 225.23158128638016
Epoch 10/30, Loss: 305.73150875693875
Epoch 11/30, Loss: 295.78003421582673
Epoch 12/30, Loss: 264.87861191598995
Epoch 13/30, Loss: 231.36500960902163
Epoch 14/30, Loss: 249.54061056438246
Epoch 15/30, Loss: 264.159214622096
Epoch 16/30, Loss: 215.31204025368942
Epoch 17/30, Loss: 266.134747505188
Epoch 18/30, Loss: 194.97364636471397
Epoch 19/30, Loss: 197.5397037706877
Epoch 20/30, Loss: 201.9171892467298
Epoch 21/30, Loss: 190.89596838700143
Epoch 22/30, Loss: 172.11866228204025
Epoch 23/30, Loss: 185.97156685277037
Epoch 24/30, Loss: 164.96476340607592
Epoch 25/30, Loss: 153.00675404699226
Epoch 26/30, Loss: 143.7648101103933
Epoch 27/30, Loss: 125.24593338

In [10]:
'''
# K 折训练

kf = KFold(n_splits=5)
num_epochs = 10
for fold, (train_index, val_index) in enumerate(kf.split(dataset)):
    print(f"Fold {fold + 1}/{kf.n_splits}")
    
    # 重新初始化模型和优化器
    model = Teeth3DCNN()
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    # 创建训练和验证数据子集
    train_subset = torch.utils.data.Subset(dataset, train_index)
    val_subset = torch.utils.data.Subset(dataset, val_index)
    train_loader = DataLoader(train_subset, batch_size=4, shuffle=True)
    val_loader = DataLoader(val_subset, batch_size=4, shuffle=False)
    
    # 训练模型
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        # 验证模型
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, targets in val_loader:
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                val_loss += loss.item()
        
        print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {running_loss / len(train_loader)}, Validation Loss: {val_loss / len(val_loader)}")
'''

'\n# K 折训练\n\nkf = KFold(n_splits=5)\nnum_epochs = 10\nfor fold, (train_index, val_index) in enumerate(kf.split(dataset)):\n    print(f"Fold {fold + 1}/{kf.n_splits}")\n    \n    # 重新初始化模型和优化器\n    model = Teeth3DCNN()\n    criterion = nn.MSELoss()\n    optimizer = optim.Adam(model.parameters(), lr=0.001)\n    \n    # 创建训练和验证数据子集\n    train_subset = torch.utils.data.Subset(dataset, train_index)\n    val_subset = torch.utils.data.Subset(dataset, val_index)\n    train_loader = DataLoader(train_subset, batch_size=4, shuffle=True)\n    val_loader = DataLoader(val_subset, batch_size=4, shuffle=False)\n    \n    # 训练模型\n    for epoch in range(num_epochs):\n        model.train()\n        running_loss = 0.0\n        for inputs, targets in train_loader:\n            optimizer.zero_grad()\n            outputs = model(inputs)\n            loss = criterion(outputs, targets)\n            loss.backward()\n            optimizer.step()\n            running_loss += loss.item()\n        \n        # 验证模型

In [11]:
# 对150个预备体进行评分
model.eval()
scores = []
with torch.no_grad():
    for i in range(len(dataset)):
        inputs, _ = dataset[i]  # 忽略目标评分，只取输入数据
        inputs = inputs.unsqueeze(0)  # 添加批次维度
        output = model(inputs)
        scores.append(output.item())

# 输出150个预备体的评分
for i, score in enumerate(scores):
    print(f'Tooth {i+1}: Score = {score}')


Tooth 1: Score = 69.09716796875
Tooth 2: Score = 67.9903564453125
Tooth 3: Score = 70.54341888427734
Tooth 4: Score = 78.18881225585938
Tooth 5: Score = 76.46572875976562
Tooth 6: Score = 66.83679962158203
Tooth 7: Score = 74.81896209716797
Tooth 8: Score = 69.56407928466797
Tooth 9: Score = 76.32901763916016
Tooth 10: Score = 75.58587646484375
Tooth 11: Score = 51.45491027832031
Tooth 12: Score = 62.63524627685547
Tooth 13: Score = 58.56817626953125
Tooth 14: Score = 65.03516387939453
Tooth 15: Score = 54.372589111328125
Tooth 16: Score = 39.654232025146484
Tooth 17: Score = 55.444496154785156
Tooth 18: Score = 60.16496658325195
Tooth 19: Score = 71.11936950683594
Tooth 20: Score = 60.5402717590332
Tooth 21: Score = 66.54730987548828
Tooth 22: Score = 63.82579803466797
Tooth 23: Score = 71.46808624267578
Tooth 24: Score = 69.31253814697266
Tooth 25: Score = 55.62440872192383
Tooth 26: Score = 60.22298812866211
Tooth 27: Score = 66.75005340576172
Tooth 28: Score = 61.48011779785156
Too

In [12]:
_, Real_Scores = read_scores(score_file_path)

for i, score in enumerate(scores):
    print(f'Tooth {i+1}: Score = {score}', f'Err = {score-Real_Scores[i]}' )

Tooth 1: Score = 69.09716796875 Err = -5.90283203125
Tooth 2: Score = 67.9903564453125 Err = -0.0096435546875
Tooth 3: Score = 70.54341888427734 Err = 0.5434188842773438
Tooth 4: Score = 78.18881225585938 Err = 9.188812255859375
Tooth 5: Score = 76.46572875976562 Err = 0.465728759765625
Tooth 6: Score = 66.83679962158203 Err = 2.8367996215820312
Tooth 7: Score = 74.81896209716797 Err = 5.818962097167969
Tooth 8: Score = 69.56407928466797 Err = -3.4359207153320312
Tooth 9: Score = 76.32901763916016 Err = -0.6709823608398438
Tooth 10: Score = 75.58587646484375 Err = -1.41412353515625
Tooth 11: Score = 51.45491027832031 Err = -2.5450897216796875
Tooth 12: Score = 62.63524627685547 Err = 0.6352462768554688
Tooth 13: Score = 58.56817626953125 Err = -0.43182373046875
Tooth 14: Score = 65.03516387939453 Err = 2.0351638793945312
Tooth 15: Score = 54.372589111328125 Err = -2.627410888671875
Tooth 16: Score = 39.654232025146484 Err = -8.345767974853516
Tooth 17: Score = 55.444496154785156 Err = 